# 🔗 On-Chain Transaction Analysis
## Step 2: Analyze On-Chain Tx Data → Spike Detection, Whale/Bot Patterns

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime
sns.set(style='whitegrid')

In [ ]:
# Load on-chain transaction data
df = pd.read_csv('../data/onchain_tx.csv', parse_dates=['timestamp'])
df['date'] = df['timestamp'].dt.date
df.head()

In [ ]:
# Aggregate transaction volume per day per token
daily_volume = df.groupby(['date', 'token'])['amount'].sum().reset_index()
daily_volume.head()

In [ ]:
# Detect spikes using rolling z-score threshold
daily_volume['rolling_mean'] = daily_volume.groupby('token')['amount'].transform(lambda x: x.rolling(3, min_periods=1).mean())
daily_volume['rolling_std'] = daily_volume.groupby('token')['amount'].transform(lambda x: x.rolling(3, min_periods=1).std())
daily_volume['z_score'] = (daily_volume['amount'] - daily_volume['rolling_mean']) / daily_volume['rolling_std']
daily_volume['spike'] = daily_volume['z_score'] > 2.0  # threshold for spike
daily_volume.head()

In [ ]:
# Whale detection: Large single transactions
whale_threshold = 5000  # adjustable
df['whale_tx'] = df['amount'] > whale_threshold
whale_tx = df[df['whale_tx']]
whale_tx.head()

In [ ]:
# Plot sample token volume with spikes
sample_token = 'ETH'
token_df = daily_volume[daily_volume['token'] == sample_token]
plt.figure(figsize=(10, 4))
sns.lineplot(data=token_df, x='date', y='amount', label='Volume')
sns.scatterplot(data=token_df[token_df['spike']], x='date', y='amount', color='red', label='Spikes')
plt.title(f'{sample_token} Volume & Spikes')
plt.xticks(rotation=45)
plt.tight_layout()
plt.show()